### Recurrent Neural Network (RNN) from Scratch
Following the [DeepLearning.AI course on Sequence models](https://www.coursera.org/learn/nlp-sequence-models).

At each time-step, the RNN tries to predict the next character given the previous ones.

- $X = x_1, ... x_{t-1}$
- $Y = x_2, ... x_t$

where $x_i$ is an input character from the training set.

In [30]:
import numpy as np

# Training data
names = open("datasets/names_ES.txt", encoding="utf-8").read().split("\n")[:4000]
pets = open("datasets/pets_ES.txt", encoding="utf-8").read().split("\n")
dinos = open("datasets/dinos.txt").read().split("\n")

print("names", len(names))
print("pets", len(pets))
print("dinos", len(dinos))

data = "\n".join(names) + "\n" + "\n".join(pets) + "\n" + "\n".join(dinos)

chars = sorted(list(set(data)))
vocab_size = len(chars)

print("vocab_size", vocab_size)
print(chars)

names 4000
pets 404
dinos 1536
vocab_size 61
['\n', ' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ō']


The model only accepts numeric input, so we could transform each character into a different number, this is called tokenization.

In [31]:
char2idx = {ch:i for i, ch in enumerate(chars)}
idx2char = {i:ch for i, ch in enumerate(chars)}
print(char2idx)

{'\n': 0, ' ': 1, 'A': 2, 'B': 3, 'C': 4, 'D': 5, 'E': 6, 'F': 7, 'G': 8, 'H': 9, 'I': 10, 'J': 11, 'K': 12, 'L': 13, 'M': 14, 'N': 15, 'O': 16, 'P': 17, 'Q': 18, 'R': 19, 'S': 20, 'T': 21, 'U': 22, 'V': 23, 'W': 24, 'X': 25, 'Y': 26, 'Z': 27, 'a': 28, 'b': 29, 'c': 30, 'd': 31, 'e': 32, 'f': 33, 'g': 34, 'h': 35, 'i': 36, 'j': 37, 'k': 38, 'l': 39, 'm': 40, 'n': 41, 'o': 42, 'p': 43, 'q': 44, 'r': 45, 's': 46, 't': 47, 'u': 48, 'v': 49, 'w': 50, 'x': 51, 'y': 52, 'z': 53, 'á': 54, 'é': 55, 'í': 56, 'ñ': 57, 'ó': 58, 'ú': 59, 'ō': 60}


We must clip the gradients to prevent it from [exploding](https://machinelearningmastery.com/exploding-gradients-in-neural-networks/) 😳.

In [32]:
def clip(gradients, maxValue):
    for gradient in gradients.values():
        np.clip(gradient, -maxValue, maxValue, gradient)

To generate the next character at time-step $t+1$ we could sample with randomness (to avoid getting always the same character) from the probability distribution obtained through softmax at $Y_t$, where each chacater has its own probability of being the next.

In [33]:
def softmax(x):
	e_x = np.exp(x - np.max(x))
	return e_x / e_x.sum(axis=0)

def sample(parameters, char_to_ix):
	Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']

	x = np.zeros((vocab_size, 1)) # input chracter one-hot
	a_prev = np.zeros((Waa.shape[1], 1)) # previous hidden state

	indices = []
	idx = -1

	counter = 0 # stop at 50 characters
	newline_character = char_to_ix['\n']

	while (idx != newline_character and counter != 50):
		# Forward propagate x
		a = np.tanh(Wax @ x + Waa @ a_prev + ba)
		y = softmax(Wya @ a + by)

		# Sample the index of a character from the probability distribution y
		idx = np.random.choice(vocab_size, p = y.ravel())
		indices.append(idx)

		# Update x and a_prev for the next iteration
		x = np.zeros((vocab_size, 1))
		x[idx] = 1
		a_prev = a

		counter += 1
	if (counter == 50):
		indices.append(char_to_ix['\n'])

	return indices

In [34]:
def rnn_step_forward(xt, a_prev, parameters):
	Wax, Waa, Wya, ba, by = parameters['Wax'], parameters['Waa'], parameters['Wya'], parameters['ba'], parameters['by']

	a = np.tanh(Waa @ a_prev + Wax @ xt + ba) 
	y = softmax(Wya @ a + by)

	return a, y

def rnn_forward(X, Y, a0, parameters):
    x, a, y_hat = {}, {}, {}
    a[-1] = np.copy(a0)
    loss = 0
    
    for t in range(len(X)):
        
        x[t] = np.zeros((vocab_size, 1)) # one-hot encode the input
        if (X[t] != None):
            x[t][X[t]] = 1
            
        a[t], y_hat[t] = rnn_step_forward(x[t], a[t - 1], parameters)
        
        loss -= np.log(y_hat[t][Y[t], 0])

    cache = (y_hat, a, x)

    return loss, cache

def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    gradients['dWya'] += dy @ a.T
    gradients['dby'] += dy
    Wya = parameters['Wya']
    Waa = parameters['Waa']
    da = Wya.T @ dy + gradients['da_next'] 
    dtanh = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['dba'] += dtanh
    gradients['dWax'] += dtanh @ x.T
    gradients['dWaa'] += dtanh @ a_prev.T
    gradients['da_next'] = Waa.T @ dtanh
    return gradients
    
def rnn_backward(X, Y, parameters, cache):
	gradients = {}

	(y_hat, a, x) = cache
	Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']

	gradients['dWax'] = np.zeros_like(Wax)
	gradients['dWaa'] = np.zeros_like(Waa)
	gradients['dWya'] = np.zeros_like(Wya)
	gradients['dba'] = np.zeros_like(ba)
	gradients['dby'] = np.zeros_like(by)
	gradients['da_next'] = np.zeros_like(a[0])

	# Backpropagate through time (!)
	for t in reversed(range(len(X))):
		dy = np.copy(y_hat[t])
		dy[Y[t]] -= 1 
		gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t - 1])

	return gradients, a

def update_parameters(parameters, gradients, lr):
	parameters['Wax'] += -lr * gradients['dWax']
	parameters['Waa'] += -lr * gradients['dWaa']
	parameters['Wya'] += -lr * gradients['dWya']
	parameters['ba']  += -lr * gradients['dba']
	parameters['by']  += -lr * gradients['dby']
	return parameters

Let be optimized, shalle we use [Gradient Descent](https://en.wikipedia.org/wiki/Gradient_descent) as our holy optimization algorithm 😔🙏.

In [35]:
def optimize(X, Y, a_prev, parameters, learning_rate=0.01):
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    clip(gradients, 5)
    
    parameters = update_parameters(parameters, gradients, learning_rate)
        
    return loss, gradients, a[len(X)-1]

In [36]:
def initialize_parameters(n_a, n_x, n_y):
    Wax = np.random.randn(n_a, n_x) * 0.01
    Waa = np.random.randn(n_a, n_a) * 0.01
    Wya = np.random.randn(n_y, n_a) * 0.01
    ba = np.zeros((n_a, 1))
    by = np.zeros((n_y, 1))
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}
    return parameters

def get_initial_loss(vocab_size, seq_length):
	return -np.log(1.0/vocab_size)*seq_length

def smooth(loss, cur_loss):
	return loss * 0.999 + cur_loss * 0.001

def get_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:] # capitalize the first character
    return txt

def model(data_x, ix_to_char, char_to_ix, num_iterations=35000, n_a=50, names_to_sample=10, verbose=False):
    n_x, n_y = vocab_size, vocab_size
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    loss = get_initial_loss(vocab_size, names_to_sample)
    
    examples = [x.strip() for x in data_x]
    np.random.shuffle(examples)

    a_prev = np.zeros((n_a, 1))
    
    last_name = "abc"
    
    for j in range(num_iterations):
        
        idx = j % len(examples)
        
        single_example = examples[idx]
        single_example_chars = [char for char in single_example]
        single_example_ix = [char_to_ix[char] for char in single_example]
        X = [None] + single_example_ix
        
        ix_newline = char_to_ix['\n']
        Y = X[1:] + [ix_newline]

        curr_loss, _, a_prev = optimize(X, Y, a_prev, parameters, learning_rate=0.01)
        
        if verbose and j in [0, len(examples) -1, len(examples)]:
            print("j = " , j, "idx = ", idx,) 
        if verbose and j in [0]:
            print("single_example =", single_example)
            print("single_example_chars", single_example_chars)
            print("single_example_ix", single_example_ix)
            print(" X = ", X, "\n", "Y =       ", Y, "\n")
        
        loss = smooth(loss, curr_loss)

        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            for _ in range(names_to_sample):
                sampled_indices = sample(parameters, char_to_ix)
                last_name = get_sample(sampled_indices, ix_to_char)
                print(last_name.replace('\n', ''))
                
            print('\n')
        
    return parameters, last_name

In [37]:
parameters, last_name = model(data.split("\n"), idx2char, char2idx, 22001, verbose=True)

j =  0 idx =  0
single_example = Coelurosauravus
single_example_chars ['C', 'o', 'e', 'l', 'u', 'r', 'o', 's', 'a', 'u', 'r', 'a', 'v', 'u', 's']
single_example_ix [4, 42, 32, 39, 48, 45, 42, 46, 28, 48, 45, 28, 49, 48, 46]
 X =  [None, 4, 42, 32, 39, 48, 45, 42, 46, 28, 48, 45, 28, 49, 48, 46] 
 Y =        [4, 42, 32, 39, 48, 45, 42, 46, 28, 48, 45, 28, 49, 48, 46, 0] 

Iteration: 0, Loss: 41.133403

HpháqñCjgHófOíOánf DGSOóDōnmQéYcKōezrQeúkkávLL YNE
QTsbávRhGXFHvgEQXeFJshorqHVBqWtJbteAkílQQEOTmWXJñO
WtUbJyePúSBhPáró QrZXASksRvqoHíqHgDMYZzXáyctkMWaup
OYswQxXsGatJkUFjHbF TSXOBñpGñGBxFRZTufoynQokQúSVqX
ÁgqMzwMYZF
HsJojgIErdbYmVmbmWOPXiyrélDjwgnFHAVOEJW voHAYgWKzñ
AssdnKBEwlSe
TiSdWaLñVbNémBōVWkRrPféwWSKévFLfñyaéGHgIéhKcPñvíZy
CiōbotGYWbvgZLYNDPVXRdyFñxwgErCgTNyIBtdIVXstáMlsKo
ÍRjTgwósTwoCYUCKK QwzóañAōzúcGohPufvtáLXa pZwFBltd




Iteration: 2000, Loss: 43.814698

Ariaerusa
Ario
Mara Ma EuoZLuna ételPJbieto Jvanola Ariiol Momla
Hnaretebdin
To
Axa raNmi
Aiólen
Zarena
Ebihen
Alo MeZurasan Olabgo Noxlapu  rna Jaa MarinanodMas


Iteration: 4000, Loss: 38.550184

Sona
Ratena Mostarra Ciana Gerestaniostonieleracamanaic
Selit
Pirico Mantetien Jola Alicio Minitna Reel
Alhel
Ante DelesUNareu Dejxeo
IAnga roVa SDlol IREAlYfOalanros
Sostenoluetorar
Noergen
Repo Mhenarodn


j =  5939 idx =  5939
j =  5940 idx =  0
Iteration: 6000, Loss: 36.284504

Emico Phuhil
Oncal
Sandelo Antrisa ElicakKilU
EOlizodonie
A
Sinlis Ks
Miria Malin
Hilel Minel
Elisio Bialll
Alvondi


Iteration: 8000, Loss: 35.126840

Garillina
Nongafar Ckin
Anne Der
Delasaurus
Agsant
Dagelsaira DepJEdm
Singy Catyrdia
Iile Cadela Frndo Gianotia
Bradopsaurus
Allcipelus


Iteration: 10000, Loss: 34.369573

JisoloD Feva
AjERPELDESABHBYADERCARIITNEQDEAMIERYENIJALARRODSMO
Neru
Jocilan Merig
Delesta Dhikayxescusaurus
Pewasta
Mandonuano Depanmo
Dezizabores
Nacila Hatre